In [520]:
import numpy as np
import random
import langchain
from langchain.llms.base import LLM
import json
import re
import sqlite3
from datetime import datetime
import pandas as np
from collections import Counter
import pandas as pd
from config import API_KEY

In [521]:
#装饰器
def truncate(a):
    return a[:-1]


def speech(func):
    def wrapper(*args, **kwargs):
#         # 在调用原始函数之前可以执行一些代码
        result = func(*args, **kwargs)
        
#         # 对原始函数的输出进行后处理
#         result = result.upper()  # 假设我们想要将输出转换为大写
#         result = truncate(result)
        
#         # 在返回结果之前可以执行一些代码
#         print("处理后的结果是:", result)
        
        print("hello,allice!")
        return result
    return wrapper

# 使用装饰器
@speech
def greet(name):
    return f"Hello, {name}!"

# 调用原始函数
print(greet("Alice"))  # 输出: Hello, ALICE!

hello,allice!
Hello, Alice!


In [522]:
#分配角色
char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}

In [523]:
#存活状态确认
def char_alive_check(aliv_list,char_dist):
    char_aliv=[]
    for i in aliv_list:
        char_aliv.append(char_dist[i])
    return set(char_aliv)

In [524]:
#角色初始定义

In [525]:
#大模型agent定义
import random
from http import HTTPStatus
import dashscope
from dashscope import Generation  # 建议dashscope SDK 的版本 >= 1.14.0

class base_llm():
    def __init__(self,system_prompt):
        self.api_key=API_KEY
        self.system_prompt=system_prompt
    def call_with_messages(self,prompt_info,temp=0.1):
        messages = [{'role': 'system', 'content': self.system_prompt},
                    {'role': 'user', 'content': prompt_info}]
        dashscope.api_key=self.api_key
        response = Generation.call(model="qwen-plus",
                                   messages=messages,
                                   # 设置随机数种子seed，如果没有设置，则随机数种子默认为1234
                                   seed=random.randint(1, 10000),
                                   temperature=temp,
                                   # 将输出设置为"message"格式
                                   result_format='message')
        if response.status_code == HTTPStatus.OK:
            return response.output.choices[0].message.content
        else:
            return ('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message))

In [526]:
# a=base_llm(system_prompt="你是一个聪明的助理")
# b=a.call_with_messages("世界上最高的山峰是什么")
# b

In [527]:
# b.output.choices[0].message.content
# def extract_content(response):
#     response=json.loads(response)
#     for choice in response.get('output', {}).get('choices', []):
#         if 'message' in choice and 'content' in choice['message']:
#             return choice['message']['content']
#     return None

# # 使用函数提取content
# content = extract_content(b)
# print(content)

In [528]:
game_intro='''《狼人杀》是一款社交推理游戏，游戏模拟了一个小村庄里人类与狼人之间的斗争。游戏目标根据不同角色有所不同，人类方需要找出并消灭所有狼人，狼人方则要尽可能消灭村民。
根据你提供的角色设置，这里有3个神职人员（1个预言家、1个女巫、1个猎人）、2个普通村民和3个狼人。下面是这些角色的基本职能和游戏的简要规则：
- 预言家：每个夜晚可以查验一名玩家的真实身份，查验这名玩家是村民方还是狼人方。
- 女巫：拥有两瓶药水，一瓶可以救活当夜被狼人杀害的玩家，另一瓶可以在夜晚毒杀一名玩家。每瓶药水只能使用一次。
- 猎人：在被消灭时（不论是白天被投票消灭还是夜里被狼人杀害），可以开枪带走一名玩家。
- 村民：没有特殊能力，主要通过讨论和投票来识别狼人。
- 狼人：每个夜晚会选择杀害一名玩家，目标是消灭所有村民，获得游戏胜利。
游戏流程大致如下：
- 游戏开始时，每位玩家随机分配一个角色，并保密自己的身份。
- 游戏分为夜晚和白天两个阶段。
- 夜晚：所有玩家闭眼，游戏主持人（通常称为法官）指挥特定角色的玩家按顺序睁眼执行他们的能力。首先是狼人相互确认身份，然后选择一个玩家作为杀害目标。随后，女巫会被告知狼人的杀害目标，决定是否使用药水，预言家可以查验一名玩家的真实身份。
- 白天：所有玩家睁眼。如果有玩家在夜里被杀害，法官会宣布该玩家死亡（除非被女巫救活）。然后所有存活的玩家进行讨论，最后进行投票，票数最多的玩家被消灭出局。如果猎人被消灭，他可以使用他的技能。
- 游戏继续进行，直到村民方或狼人方达成胜利条件。村民方胜利条件是所有狼人被消灭；狼人方胜利条件是2个普通村民全被消灭，或3个神职人员全被消灭。
'''
char_intro_werewolf='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是狼人，游戏开始时你知道你的狼人队友身份，你不知道其他玩家的身份。
    你和你的狼人队友属于狼人阵营，你们的胜利条件是消灭所有的村民或者所有的神。你的特殊能力是和狼人队友在晚上讨论后消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚和狼队友讨论后投票杀掉一个玩家。
    2.白天发表言论，保护自己和狼人队友，让大家投票杀掉一个玩家。
'''
char_intro_seer='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是预言家。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是可以在夜晚验明一个玩家的真实身份。
    因此在游戏中你的工作包括以下内容：
    1.通过你的预言家身份，在夜晚查验某个玩家的真实身份。
    2.在白天，把你得到的信息告诉大家，如果你查验玩家的真实身份属于人类阵营，通过发言保护这个玩家不被投票消灭；如果你查验玩家的真实身份是狼人，引导大家用可能的方式消灭他。
'''
char_intro_witch='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是女巫。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一瓶起死回生的解药和一瓶杀人的毒药，你可以在夜晚决定是否用解药救下被狼人杀掉的玩家，或者用毒药杀掉一个玩家，注意一局游戏中两种药都只能用一次。
    因此在游戏中你的工作包括以下内容：
    1.在夜晚，根据你对当前局势的判断，决定是否用解药救下某个被杀掉的玩家，或者是否用毒药杀掉某个玩家。
    2.在白天发表言论，根据你观察到的信息决定保护人类阵营的玩家，同时帮大家找到狼人并投票杀掉狼人玩家。
'''
char_intro_hunter='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是猎人。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你的特殊能力是有一把猎枪，当你以任意方式死亡时，你可以立即决定是否用猎枪消灭一名玩家。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
    2.当你死亡时，根据你对当前局势的判断决定是否杀掉某个你认为一定是狼人的玩家，或者不开枪以免误伤人类阵营队友；
'''
char_intro_villager='''你在玩一局狼人杀游戏，这是一个8人局，其中包含3个狼人、2个村民和3个神职人员（预言家、女巫和猎人），这局中你抽到的角色是村民。
    你和所有神以及村民属于人类阵营，你们的胜利条件是消灭所有的狼人。你没有任何特殊能力。
    因此在游戏中你的工作包括以下内容：
    1.在白天发表言论，根据你的推理保护人类阵营玩家，找到狼人玩家并投票消灭他们。
'''

In [529]:
def char_info_initialize(char_dist):
    char_info_dic={}
    for i in char_dist.keys():
        if char_dist[i]=='villager':
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是村民。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='seer':
            info=game_intro+char_intro_seer+"你是%d号玩家，你的身份是预言家。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='witch':
            info=game_intro+char_intro_witch+"你是%d号玩家，你的身份是女巫。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='hunter':
            info=game_intro+char_intro_hunter+"你是%d号玩家，你的身份是猎人。\n"%(i)
            char_info_dic[i]=info
        elif char_dist[i]=='werewolf':
            werewolf_list=[key for key,value in char_dist.items() if value=='werewolf']
            werewolf_list.remove(i)
            info=game_intro+char_intro_villager+"你是%d号玩家，你的身份是狼人，你的狼人队友包括"%(i)
            for j in werewolf_list:
                info+="%d号玩家、"%(j)
            info=info[:-1]+"。\n"
            char_info_dic[i]=info
    return char_info_dic

In [530]:
#狼人
class werewolf():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='werewolf'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def kill_speech(self,strategy=''):     #杀人发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考应该消灭几号玩家，并在和其他狼人的讨论中发言。
        请仔细思考，然后把发言内容写到【发言内容】中,发言内容请简洁，必须控制在150字内。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('夜晚狼人杀人发言',self.num,self.speaker,res)
        return res
    def kill_vote(self,strategy=''):     #杀人投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，作为一个狼人玩家，请仔细思考决定投票消灭几号玩家,并给出投票结果的玩家编号,记住你只能消灭1位玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('夜晚狼人杀人投票',self.num,self.speaker,res)
        return res
    @speech
    def speech(self,strategy='1.为了提高狼人队友的存活率，不要在发言中质疑狼人阵营玩家的发言，减少大家投票消灭狼人阵营玩家的概率；2.如果你之前有人类阵营的玩家发言，尝试通过质疑他的发言内容引导大家投票消灭他。'):    #白天发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】，发表你的看法，误导人类阵营。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy='不要投票选择自己阵营的狼人玩家消灭。'):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你从你的角色和胜利条件出发，根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭几号玩家才能获胜,记住你只能消灭1位玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [531]:
#seer
class seer():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='seer'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def char_val(self,strategy=''):     #验身份
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，你可以选择1个玩家获得其真实身份信息，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考验证哪个嫌疑最大的玩家，并给出玩家编号,记住你只能验证一位玩家的身份。
        请仔细思考中，然后把验身份的对象玩家编号写到【玩家编号】中。
        【玩家编号】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【玩家编号】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('预言家验证身份',self.num,self.speaker,res)
        return res
    @speech
    def speech(self,strategy=''):    #白天发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你从你的角色和胜利条件出发，根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭几号玩家,记住你只能消灭1位玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [532]:
#witch
class witch():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='witch'
        self.logger=logger
        self.poison=1
        self.antidote=1
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def use_poison(self,strategy=''):     #毒药
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间，请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用女巫的毒药消灭一名玩家，以及消灭几号玩家,记住你只能消灭1位玩家。
        请仔细思考，然后把你的决定写到【毒药决策】中。
        【毒药决策】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【毒药决策】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('毒药决策',self.num,self.speaker,res)
        return res
    def use_antidote(self,killed_player,strategy="1.如果被消灭的玩家是你自己，一定要使用解药；2.如果你确定被消灭的玩家是预言家，可以考虑使用解药；3.根据胜利条件和当前剩余玩家的角色判断即将输掉比赛，请使用解药解救该玩家。"):     #解药
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是晚上时间,%d号玩家被狼人消灭,请你根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考，并是否使用解药复活他。
        请仔细思考，然后把你的决定写到【解药决策】中。
        【解药决策】:'''%(killed_player)
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【解药决策】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('解药决策',self.num,self.speaker,res)
        return res
    @speech
    def speech(self,strategy=''):    #发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你从你的角色和胜利条件出发，根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭几号玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [533]:
#villager
class villager():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='villager'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    @speech
    def speech(self,strategy=''):    #发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你从你的角色和胜利条件出发，根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭几号玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [534]:
#hunter
class hunter():
    def __init__(self,char_info,num,logger):
        self.char_info=char_info
        self.llm=base_llm(self.char_info)
        self.num=num
        self.speaker='hunter'
        self.logger=logger
    def get_cur_state(self):
        return self.logger.get_cur_log(self.speaker)
    def vengence(self,strategy=''):     #复仇子弹
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在你即将被消灭，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】，请仔细思考是否使用猎人的猎枪复仇消灭一名玩家，以及消灭几号玩家,记住你只能消灭1位玩家。
        请仔细思考，然后把你的决定写到【猎枪决策】中。
        【猎枪决策】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【猎枪决策】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('猎枪决策',self.num,self.speaker,res)
        return res
    @speech
    def speech(self,strategy=''):    #发言
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天时间轮到你发言，请你从你的角色和胜利条件出发，根据【游戏记录】中的信息，结合【游戏策略】和你获得的信息，仔细思考如何发言才能帮助人类阵营获胜，并发表你的看法。
        请仔细思考，然后把发言内容写到【发言内容】中。
        【发言内容】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【发言内容】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天发言',self.num,self.speaker,res)
        return res
    def day_vote(self,strategy=''):    #白天投票
        cur_state=self.get_cur_state()
        prompt=cur_state+strategy+'''现在是白天投票时间，请你从你的角色和胜利条件出发，根据【游戏记录】中今天的对话和历史信息，结合【游戏策略】，仔细思考投票决定消灭几号玩家,记住你只能消灭1位玩家。
        请仔细思考，然后把投票结果写到【投票结果】中。
        【投票结果】:'''
        result=self.llm.call_with_messages(prompt)
#         res=re.search(r"【投票结果】(.*)",result).group(1)[1:]
        res=result
        self.logger.add_log('白天投票',self.num,self.speaker,res)
        return res

In [535]:
#game_endcheck
def endcheck(aliv_list,char_dist,logger):
    translator={'werewolf':'狼人',
                    'seer':'预言家',
                    'witch':'女巫',
                    'villager':'村民',
                    'hunter':'猎人',
                    'moderater':'裁判'}
    char_aliv=char_alive_check(aliv_list,char_dist)
    if "villager" not in char_aliv:
        logger.add_log('观察日志',-1,'moderater','游戏结束,普通村民全被消灭，狼人阵营胜利')
        log_info="本局游戏玩家角色:"
        for i in char_dist.keys():
            log_info+= str(i)+"号玩家-"+translator[char_dist[i]]+'、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        log_info="剩余存活玩家:"
        for i in aliv_list:
            log_info+= str(i)+"号玩家("+translator[char_dist[i]]+')、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        return 1
    elif char_aliv=={"werewolf","villager"}:
        logger.add_log('观察日志',-1,'moderater','游戏结束,神职人员全被消灭，狼人阵营胜利')
        log_info="本局游戏玩家角色:"
        for i in char_dist.keys():
            log_info+= str(i)+"号玩家-"+translator[char_dist[i]]+'、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        log_info="剩余存活玩家:"
        for i in aliv_list:
            log_info+= str(i)+"号玩家("+translator[char_dist[i]]+')、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        return 1
    elif "werewolf" not in char_aliv:
        logger.add_log('观察日志',-1,'moderater','游戏结束,狼人全被消灭，人类阵营胜利')
        log_info="本局游戏玩家角色:"
        for i in char_dist.keys():
            log_info+= str(i)+"号玩家-"+translator[char_dist[i]]+'、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        log_info="剩余存活玩家:"
        for i in aliv_list:
            log_info+= str(i)+"号玩家("+translator[char_dist[i]]+')、'
            log_inf=log_info[:-1]+'。'
        logger.add_log('观察日志',-1,'moderater',log_info)
        return 2
    else:
        return 0    

In [536]:
#动作信息提取
input_2_json_llm=base_llm("你是一个从对话内容提取json字符串的小助手，可以根据提示从对话中精准提取信息，并以规定的json格式返回")
def kill_vote_extr(llm,input):
    prompt='''请根据狼人的投票内容提取他的投票消灭的对象，如果玩家指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为投了弃权票，以这种格式返回结果:{'是否消灭':'否'}。
    例1：由于提供的狼人【投票结果】中并未包含具体的投票对象，而是描述了一般情况下狼人的游戏策略，因此无法直接提取出{'是否消灭':'是','消灭玩家':x}这样的信息。如果需要提取实际的投票信息，请提供包含具体投票目标的对话内容。
    【提取结果】:{'是否消灭':'否'}
    例2：狼人的【投票结果】:【投票结果】: 4号玩家

根据游戏记录，4号玩家的积极参与和5号玩家的验人结果已经引起了注意，这可能意味着他正在积极寻找狼人。如果他是神职，他的出局将立即揭示他的身份，这可能会导致狼人阵营的暴露。6号玩家的沉默虽然可疑，但如果我们今晚投票给他，如果他不是守卫，那么我们可能会失去一个潜在的狼人，而不会立即暴露狼人身份。

考虑到这些因素，我选择投票给4号玩家。如果他是神职，他的出局将消除一个对狼人威胁较大的角色，而且他的死亡可能会引发人类阵营的混乱，因为他们会试图找出是谁杀了他。如果4号是村民，那么我们只是提前淘汰了一个可能的威胁，而不会立即暴露狼人身份。这种策略旨在制造更多混乱，同时降低我们被发现的风险。
    【提取结果】:{'是否消灭':'是','消灭玩家':4}
    例3：狼人的【投票结果】:【投票结果】: 0号玩家
    【提取结果】:{'是否消灭':'是','消灭玩家':0}
    #####################################
    当前狼人的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def day_vote_extr(llm,input):
    prompt='''请根据当前玩家的投票内容提取他的投票消灭的对象，如果玩家指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为投了弃权票，以这种格式返回结果:{'是否消灭':'否'}。
    例1：玩家的【投票结果】:在白天的讨论中，虽然5号玩家被多次提及，我需要保持谨慎，避免过早暴露自己。0号玩家在发言中特别提到了我，这让我有些怀疑他的动机。狼人可能会试图引导村民对我产生怀疑，以转移对其他狼人成员的注意力。因此，我选择不投票给5号玩家，而是将注意力转向0号玩家，他的发言和对我行为的关注让我觉得他可能是一个狼人。当然，这只是一个策略，我需要在接下来的讨论中收集更多信息来确认我的猜测。
    【提取结果】:{'是否消灭':'是','消灭玩家':0}
    例2：玩家的【投票结果】:由于目前没有确凿的证据指向某个玩家是狼人，而且1号玩家的发言虽然试图展现新人的紧张，但也有玩家表示了他的警觉，我作为女巫需要更加谨慎。在这种情况下，我倾向于观察和收集更多的信息，避免在初期就草率投票。因此，我选择弃权，不进行投票。这样可以避免因误判而失去一位可能的队友，同时也能保持游戏的平衡，等待更多的线索出现。当然，如果其他玩家提供了有力的证据，我会重新考虑我的立场。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【投票结果】:%s。
    【提取结果】:
    '''%(input)
    print("决策过程:")
    print(input)
    result=llm.call_with_messages(prompt)
    print("投票文本:")
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def seer_val_extr(llm,input):
    prompt='''请根据玩家的选择决策提取他的投票验证身份的对象，如果玩家明确指定了被验证玩家的编号，以类似以下格式返回结果:{'是否验证':'是','验证玩家':4};如果没有指明唯一的验证对象，则认为投了弃权票，以这种格式返回结果:{'是否验证':'否'}。
    例1：玩家的【选择决策】:由于这是游戏的第1个晚上，没有之前的讨论和投票信息可供参考，所以选择验人可能会比较随机。不过，通常情况下，玩家会优先考虑验那些在开局时表现得异常活跃或者沉默不语的玩家，因为他们可能是试图掩饰自己身份的狼人。

另一种策略是验与自己关系最近的玩家，比如紧挨着你的2号玩家。这样可以初步了解你周围的安全情况。因此，作为1号玩家，我建议验2号玩家的身份。

【玩家编号】: 2
    【提取结果】:{'是否验证':'是','验证玩家':2}
    例2：在这种情况下，由于是游戏的第一夜，还没有足够的信息来判断谁最可疑。因此，选择查验的对象通常是随机的，或者基于初步观察和直觉。一种常见的策略是优先查验有特殊身份的玩家，比如女巫或猎人，但如果无法确定，可以选择任意一个玩家开始。

为了给出一个具体的答案，我将选择查验一个玩家作为示例，但请注意，在实际游戏中这一步往往是基于直觉或随机选择的，因为还没有任何线索指向特定的玩家。

【玩家编号】: 1
    【提取结果】:{'是否验证':'是','验证玩家':1}
    #####################################
    当前玩家的【选择决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否验证']=='是':
        return kill_res['验证玩家']
    else:
        return -1
def witch_poison_extr(llm,input):
    prompt='''请根据当前玩家的毒药决策提取他的决定用毒药消灭的对象，如果玩家确定要使用毒药，并明确指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为不使用毒药消灭玩家，以这种格式返回结果:{'是否消灭':'否'}。
    例1：玩家的【毒药决策】:根据当前的局面，我想用毒药毒死3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【毒药决策】:2号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':2}
    例3:在目前的情况下，由于信息有限，我没有足够的理由直接使用毒药。因此，我的初步决定是**不使用毒药**，以待更多信息出现或出现更明确的狼人目标。然而，如果游戏中出现明显的狼人行为或有玩家的行为让我高度怀疑其为狼人，我会重新考虑这个决策。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【毒药决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1
def witch_antidote_extr(llm,input):
    prompt='''请根据当前玩家的解药决策提取是否解救的决定，如果玩家明确表示愿意使用解药，以类似以下格式返回结果:{'是否解救':'是'}；否则玩家没有明确表示要使用解药，以下面格式返回结果:{'是否解救':'否'}。
    例1：玩家的【解药决策】:根据当前的局面，我不想用解药救下这个人。
    【提取结果】:{'是否解救':'否'}
    例2：玩家的【解药决策】:救。
    【提取结果】:{'是否解救':'是'}
    #####################################
    当前玩家的【解药决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    return kill_res['是否解救']
def hunter_vegence_extr(llm,input):
    prompt='''请根据当前玩家的猎枪决策提取他的决定用猎枪复仇消灭的对象，如果玩家确定要使用猎枪，并明确指定了被消灭玩家的编号，以类似以下格式返回结果:{'是否消灭':'是','消灭玩家':4}；如果没有指明唯一的消灭对象，则认为不使用猎枪消灭玩家，以这种格式返回结果:{'是否消灭':'否'}。
    例1：玩家的【猎枪决策】:根据当前的局面，我想用猎枪带走3号玩家。
    【提取结果】:{'是否消灭':'是','消灭玩家':3}
    例2：玩家的【猎枪决策】:由于缺乏具体信息，我只能做出一个假设性的决定。在实际情况中，我会需要更多的游戏进程信息来进行决策。如果我是被狼人直接杀害的，而我之前没有任何信息，我的【猎枪决策】可能是不开枪，因为这样更谨慎，避免误伤好人。但如果是游戏实际进行中，这个决定应基于实际发生的事件和玩家的言行。
    【提取结果】:{'是否消灭':'否'}
    #####################################
    当前玩家的【猎枪决策】:%s。
    【提取结果】:
    '''%(input)
    print(input)
    result=llm.call_with_messages(prompt)
    result=re.search(r'\{(.*?)\}', result)[0]
    print(result)
    kill_res=eval(result)
    if kill_res['是否消灭']=='是':
        return kill_res['消灭玩家']
    else:
        return -1

In [537]:
class game_logger():
    def __init__(self,db_name):
        self.conn=sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.table_name="game_"+datetime.now().strftime("%Y%m%d%H%M%S")
        create_table_sql = f'''CREATE TABLE IF NOT EXISTS {self.table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        type TEXT NOT NULL,
        player INT NOT NULL,
        speaker TEXT NOT NULL,
        content TEXT NOT NULL
            )
        '''
        self.cursor.execute(create_table_sql)
        self.conn.commit()
    def add_log(self,log_type,num,speaker,content):
        insert_sql = f"INSERT INTO '{self.table_name}' (type,player, speaker,content) VALUES (?, ?, ?,?)"
        self.cursor.execute(insert_sql, (log_type,num, speaker,content))
        self.conn.commit()
        return 0
    def get_cur_log(self,speaker):
        translator={'werewolf':'狼人',
                    'seer':'预言家',
                    'witch':'女巫',
                    'villager':'村民',
                    'hunter':'猎人',
                    'moderater':'裁判'}
        if speaker=="werewolf":
            allowed_speaker={'werewolf':['夜晚狼人杀人发言','白天发言'],
                            'seer':['白天发言'],
                            'witch':['白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        elif speaker=="seer":
            allowed_speaker={'werewolf':['白天发言'],
                            'seer':['预言家验证身份','白天发言'],
                            'witch':['白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        elif speaker=="witch":
            allowed_speaker={'werewolf':['白天发言'],
                            'seer':['白天发言'],
                            'witch':['解药决策','毒药决策','白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        else:
            allowed_speaker={'werewolf':['白天发言'],
                            'seer':['白天发言'],
                            'witch':['白天发言'],
                            'villager':['白天发言'],
                            'hunter':['白天发言'],
                            'moderater':['观察日志']}
            df = pd.read_sql_query(f"SELECT * FROM '{self.table_name}' ", self.conn)
            night=1
            day=0
            history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
            for index, row in df.iterrows():
                if row['type'] in allowed_speaker[row['speaker']]:
                    if row['speaker']=='moderater':
                        if row['content']=='夜晚结束':
                            day+=1
                            history_log+=f"第 %d 天:\n"%day
                        elif row['content']=='白天结束':
                            night+=1
                            history_log+=f"第 %d 夜:\n"%night
                        else:
                            formatted_log = f"发生事件: {row['content']}"
                            history_log+=formatted_log +"\n"
                    else:
                        formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                        history_log+=formatted_log +"\n"
            return history_log
        def close_connection(self):
            self.cursor.close()
            self.conn.close()
            return 0

In [538]:
def game_init():
    logger=game_logger('werewolf.db')
    char_quota={"seer":1,"witch":1,"hunter":1,"villager":2,"werewolf":3}
    numbers=list(range(8))
    aliv_list=list(range(8))
    random.shuffle(numbers)
    char_dist={}
    char_dist[numbers[0]]="seer"
    char_dist[numbers[1]]="witch"
    char_dist[numbers[2]]="hunter"
    char_dist[numbers[3]]="villager"
    char_dist[numbers[4]]="villager"
    char_dist[numbers[5]]="werewolf"
    char_dist[numbers[6]]="werewolf"
    char_dist[numbers[7]]="werewolf"
    char_info=char_info_initialize(char_dist=char_dist)
    char_instances={}
    for i in char_dist.keys():
        if char_dist[i]=="werewolf":
            char_instances[i]=werewolf(char_info[i],i,logger)
        elif char_dist[i]=="seer":
            char_instances[i]=seer(char_info[i],i,logger)
        elif char_dist[i]=="witch":
            char_instances[i]=witch(char_info[i],i,logger)
        elif char_dist[i]=="villager":
            char_instances[i]=villager(char_info[i],i,logger)
        elif char_dist[i]=="hunter":
            char_instances[i]=hunter(char_info[i],i,logger)
    log_info="游戏开始，当前存活玩家:"
    for i in aliv_list:
        log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
    logger.add_log('观察日志',-1,'moderater',log_info)
    return aliv_list,char_dist,char_instances,logger

In [539]:
def night_werewolf_kill(werewolves):
    print("夜晚狼人开始行动：")
    for werewolf in werewolves:
        res=werewolf.kill_speech()
        print(f"狼人{werewolf.num}号发言:"+res)
    votes=[]
    for werewolf in werewolves:
        res = werewolf.kill_vote()
        vote_res=kill_vote_extr(input_2_json_llm,res)
        votes.append(vote_res)
    print("vote result:")
    print(votes)
    print(Counter(votes).most_common(1)[0][0])
    return Counter(votes).most_common(1)[0][0]
def night_witch(witch,killed_num):
    print("女巫开始行动：")
    killed_rescue= -1
    poison_kill=-1
    if witch.antidote==1 and killed_num>-1:
        res=witch.use_antidote(killed_num)
        antidote_res=witch_antidote_extr(input_2_json_llm,res)
        if antidote_res == '是':
            witch.antidote=0
            killed_rescue = killed_num
    if witch.poison==1:
        res=witch.use_poison()
        poison_res=witch_poison_extr(input_2_json_llm,res)
        if poison_res != -1:
            witch.poison=0
            poison_kill=poison_res
    return killed_rescue,poison_kill         
def night_seer(seer,num,char_dist,game_logger):
    print("预言家开始行动：")
    res=seer.char_val()
    val_res=seer_val_extr(input_2_json_llm,res)
#     translator={'werewolf':'狼人',
#                 'seer':'预言家',
#                 'witch':'女巫',
#                 'villager':'村民',
#                 'hunter':'猎人'}    
    if val_res==-1:
        content="预言家今夜选择不验证任何人的身份"
        print(content)
        game_logger.add_log('预言家验证身份',num,'seer',content)
    else:
        if char_dist[val_res] == 'werewolf':
            content=f"{val_res}号玩家身份被验证是狼人，属于狼人阵营"
            print(content)
        else:
            content=f"{val_res}号玩家身份被验证属于人类阵营"
            print(content)
        game_logger.add_log('预言家验证身份',num,'seer',content)
def hunter_vengence(hunter):
    vengence_kill=-1
    print("猎人即将死亡，开始复仇决策：")
    res=hunter.vengence()
    print(res)
    vengence_res=hunter_vegence_extr(input_2_json_llm,res)
    if vengence_res!= -1:
        vengence_kill=vengence_res
    return vengence_kill
def day_kill(alive_list,char_instances,game_logger):
    print("白天讨论开始：")
    alive_list.sort()
    for i in alive_list:
        res=char_instances[i].speech()
        print(f"{i}:{res}")
    votes=[]
    vote_gesture=dict()
    for i in alive_list:
        res=char_instances[i].day_vote()
        vote_res=day_vote_extr(input_2_json_llm,res)
        vote_gesture[i]=vote_res
        votes.append(vote_res)
    for j in vote_gesture.keys():
        log_info=f'{j}号玩家投票:{vote_gesture[j]}号玩家'
        game_logger.add_log('观察日志',-1,'moderater',log_info)
    return Counter(votes).most_common(1)[0][0]
def kill_result(alive_list,kill_list):
    for i in kill_list:
        if i in alive_list:
            alive_list.remove(i)
        else:
            print(f"{i}号玩家已经死亡，不能再次消灭！")
    return alive_list

In [540]:
#night process
def night(game_logger,alive_list,char_dist,char_instances,round_count):
    #初始化,确认角色分布
    alive_char=char_alive_check(alive_list,char_dist)
    kill_list=[]
    rescue_list=[]
    #狼人杀人
    werewolves=[]
    for i in alive_list:
        if char_dist[i] == "werewolf":
            werewolves.append(char_instances[i])
    werewolf_killed=night_werewolf_kill(werewolves)
    if werewolf_killed>-1:
        kill_list.append(werewolf_killed)
    #女巫救人/毒人
    if 'witch' in alive_char:
        for i in char_dist.keys():
            if char_dist[i] == 'witch':
                witch=char_instances[i]
        rescue,poison=night_witch(witch,werewolf_killed)
        if poison>-1:
            kill_list.append(poison)
        if rescue>-1:
            rescue_list.append(rescue)
    #预言家验人
    if 'seer' in alive_char:
        for i in char_dist.keys():
            if char_dist[i] == 'seer':
                seer=char_instances[i]
                num_seer=i
        night_seer(seer,num_seer,char_dist,game_logger)
    #夜晚结算
    for i in rescue_list:
        if i in kill_list:
            kill_list.remove(i)
    for j in kill_list:
        if char_dist[j] == 'hunter':
            hunter= char_instances[j]
            vengence_res=hunter_vengence(hunter)
            if vengence_res>-1:
                kill_list.append(vengence_res)
                log_info=f'猎人开枪带走了{vengence_res}号玩家'
                game_logger.add_log('观察日志',-1,'moderater',log_info)
    for k in kill_list:
        if k in alive_list:
            alive_list.remove(k)
    print("夜晚死亡玩家：")
    print(kill_list)
    if len(kill_list)>0:
        log_info="今晚死亡玩家:"
        for i in kill_list:
            log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
        game_logger.add_log('观察日志',-1,'moderater',log_info)
    else:
        game_logger.add_log('观察日志',-1,'moderater','今晚无人死亡')
    print("剩余玩家：")
    print(alive_list)
    log_info="今夜剩余存活玩家:"
    for i in alive_list:
        log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
    game_logger.add_log('观察日志',-1,'moderater',log_info)
    game_res= endcheck(alive_list,char_dist,game_logger)
    game_logger.add_log('观察日志',-1,'moderater','夜晚结束')
    return game_res,alive_list

In [541]:
#day process
def day(game_logger,alive_list,char_dist,char_instances,round_count):
    #初始化
    alive_char=char_alive_check(alive_list,char_dist)
    #发言
    day_killed=day_kill(alive_list,char_instances,game_logger)
    #结算
    kill_list=[]
    if day_killed>-1:
        kill_list.append(day_killed)
        if char_dist[day_killed] == 'hunter':
            hunter= char_instances[day_killed]
            vengence_res=hunter_vengence(hunter)
            if vengence_res>-1:
                kill_list.append(vengence_res)
                log_info=f'猎人开枪带走了{vengence_res}号玩家'
                game_logger.add_log('观察日志',-1,'moderater',log_info)
    for i in kill_list:
        if i in alive_list:
            alive_list.remove(i)
    print("白天死亡玩家：")
    print(kill_list)
    if len(kill_list)>0:
        log_info="今天死亡玩家:"
        for i in kill_list:
            log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
        game_logger.add_log('观察日志',-1,'moderater',log_info)
    else:
        game_logger.add_log('观察日志',-1,'moderater','今天无人死亡')
    print("剩余玩家：")
    print(alive_list)
    log_info="今天剩余存活玩家:"
    for i in alive_list:
        log_info+= str(i)+"号玩家、"
        log_inf=log_info[:-1]+'。'
    game_logger.add_log('观察日志',-1,'moderater',log_info)
    game_res= endcheck(alive_list,char_dist,game_logger)
    game_logger.add_log('观察日志',-1,'moderater','白天结束')
    return game_res,alive_list

In [542]:
#main process
def game_start():
    alive_list,char_dist,char_instances,logger=game_init()
    game_result=0
    round_counter=0
    while round_counter<4:
        game_res,alive_list=night(logger,alive_list,char_dist,char_instances,round_counter)
        game_result=game_res
        if game_result>0:
            print(alive_list)
            print(char_dist)
            break
        else:  
            game_res,alive_list=day(logger,alive_list,char_dist,char_instances,round_counter)
            game_result=game_res
            if game_result>0:
                print(alive_list)
                print(char_dist)
                break
            else:
                round_counter+=1
                continue
    print(game_result)
    print(alive_list)
    print(char_dist)
    return game_result

In [543]:
def speech(func):
    def wrapper(*args, **kwargs):
#         # 在调用原始函数之前可以执行一些代码
        result = func(*args, **kwargs)
        
#         # 对原始函数的输出进行后处理
#         result = result.upper()  # 假设我们想要将输出转换为大写
#         result = truncate(result)
        
#         # 在返回结果之前可以执行一些代码
#         print("处理后的结果是:", result)
        
        return result
    return wrapper

In [544]:
res_list=[]
for i in range(1):
    a=game_start()
    res_list.append(a)

夜晚狼人开始行动：
狼人1号发言:【发言内容】: 我们需要选择一个看起来可能是神职的玩家作为目标，优先考虑那些可能对局势有重大影响的角色。从目前的情况来看，我们可以先针对那些发言比较积极或者看起来可能带有重要技能的玩家。建议优先考虑3号或5号玩家，因为他们昨晚的行为显得有些可疑。我提议我们今晚先对3号玩家进行行动。 

（注：这里假定3号玩家在前一晚的行为引起了注意，实际游戏中应根据具体情况调整目标。）
狼人2号发言:【发言内容】: 同意1号的分析，3号玩家行为确实可疑，可能是神职。优先解决潜在威胁是明智之举。支持对3号玩家采取行动。
狼人6号发言:【发言内容】: 同意1号和2号的分析，3号玩家行为确实可疑。为了减少潜在威胁，我们应该优先对他采取行动。确认对3号玩家进行行动。
【投票结果】: 3号玩家
{'是否消灭':'是','消灭玩家':3}
【投票结果】: 3号玩家
{'是否消灭':'是','消灭玩家':3}
【投票结果】: 3号玩家
{'是否消灭':'是','消灭玩家':3}
vote result:
[3, 3, 3]
3
女巫开始行动：
【解药决策】: 由于被消灭的玩家是自己（3号玩家），根据游戏策略的第一条指示，“如果被消灭的玩家是你自己，一定要使用解药”，因此我将使用解药自救。这样可以保持我的存活，继续为村民阵营提供帮助。 

使用解药。
{'是否解救':'是'}
【毒药决策】: 在第一晚，由于我刚刚使用了解药自救，尚不清楚其他人的具体身份和局势。在这种情况下，盲目使用毒药可能会误杀无辜的村民或神职人员，反而削弱了我们阵营的力量。因此，在局势还不明朗的时候，谨慎起见，我决定今晚不使用毒药。

不使用毒药。
{'是否消灭':'否'}
预言家开始行动：
在这种情况下，由于是第一晚，还没有足够的信息来判断谁最可疑。通常来说，在早期阶段，任何玩家都有可能是狼人。为了获取有用的信息，我会选择查验一个看起来比较关键或者直觉上感觉可疑的玩家。这里并没有前期的行为可以参考，因此选择可以基于位置或者其他非行为因素。

一种常见的策略是在早期查验可能的关键角色，比如可能会考虑查验一个玩家来试图尽早找出女巫或猎人，但在这个阶段，这样的策略未必有效，因为还不知道哪些玩家更有可能是关键角色。因此，我会选择查验一个中间编号的玩家，以避免一开始就针对边缘位置的玩家。

【玩家编号】

决策过程:
【投票结果】: 我将投票给0号玩家。

理由如下：
1. **过于自信和流畅的发言**：多位玩家（如1号、2号、6号和7号玩家）都提到0号玩家的发言显得过于自信和流畅，给人一种刻意营造的感觉。这种表现可能是在试图引导舆论，这是狼人常见的策略之一。
2. **多次被质疑**：0号玩家的发言引起了多位玩家的质疑，特别是6号玩家明确表示对0号玩家的发言感到怀疑，认为其过于刻意。
3. **预言家的建议**：7号玩家作为预言家，查验了3号玩家并确认她是人类阵营，这意味着3号玩家不是狼人。7号玩家也提到了需要关注那些试图主导讨论进程的玩家，而0号玩家正是这样一位试图主导讨论的玩家。
4. **逻辑推理与直觉**：虽然0号玩家强调逻辑推理的重要性，但多位玩家指出在这个游戏中直觉和观察同样重要。0号玩家的异常镇定和自信让许多人产生了疑虑。

基于以上几点，我认为0号玩家有很大的嫌疑是狼人，因此我选择投票给0号玩家。
投票文本:
{'是否消灭':'是','消灭玩家':0}
决策过程:
【投票结果】: 我将投票给0号玩家。

理由如下：
- 0号玩家的发言虽然逻辑清晰，但显得过于自信和流畅，给人一种刻意营造的感觉。
- 多位玩家（包括1号、2号和6号玩家）都对0号玩家的发言表示出了疑虑，认为其表现过于镇定，这可能是一种试图主导讨论的行为。
- 在狼人杀游戏中，过于自信和试图主导讨论的玩家往往是狼人的典型表现，因为他们试图混淆视听，引导村民做出错误的决策。
- 作为村民，我们需要谨慎对待那些试图用逻辑掩盖真实目的的玩家，0号玩家的发言符合这一特征。

基于以上分析，我认为0号玩家存在较大嫌疑，因此选择投票给他。
投票文本:
{'是否消灭':'是','消灭玩家':0}
决策过程:
【投票结果】: 我将投票给0号玩家。

理由如下：
- 0号玩家的发言虽然逻辑清晰，但显得过于自信和流畅，给人一种刻意营造的感觉。
- 多位玩家（包括1号、2号和6号玩家）都对0号玩家的发言表示出了疑虑，认为其表现过于镇定，这可能是一种试图主导讨论的行为。
- 在狼人杀游戏中，过于自信和试图主导讨论的玩家往往是狼人的典型表现，因为他们试图混淆视听，引导村民做出错误的决策。
- 作为村民，我们需要谨慎对待那些试图用逻辑来引导讨论的玩家，因为真正的村民往往会有更多的犹豫和不确定性。

基于以上分析，我认

hello,allice!
5:【发言内容】:

大家早上好，首先感谢2号、3号和4号玩家的精彩发言。昨晚我们失去了7号玩家和1号玩家，这对我们的团队来说确实是一个巨大的打击。我们需要更加紧密地合作，共同找出隐藏在我们中间的狼人。

昨晚的情况非常复杂，7号玩家自称预言家并查验了3号玩家，结果表明3号玩家是人类。然而，7号玩家不幸遇害，这让我们不得不重新评估局势。我认为7号玩家可能是真的预言家，因为如果是狼人，他们通常不会轻易暴露自己的身份。因此，我们需要重新审视其他玩家的行为。

1号玩家昨晚也被狼人杀害，这让我们不得不重新考虑1号玩家的身份。或许他是真正的村民，而我们误判了他的立场。我们需要更加谨慎地对待每一位玩家的言行。

昨晚的情况表明，我们需要更加谨慎地对待每一位玩家的言行。特别是那些试图主导讨论进程的玩家，我们需要更加敏锐地捕捉这些细节。昨晚7号玩家的死亡让我们不得不重新评估局势，我们需要更加紧密地合作，共同找出隐藏在我们中间的狼人。

在这里，我想提出几点建议：

1. **保持冷静和理性**：我们需要保持冷静，不要被情绪所左右。狼人可能会试图用看似合理的论点来掩盖他们的真正目的，我们需要更加敏锐地捕捉这些细节。
2. **综合考虑每个人的言行**：我们需要综合考虑每个人的言行举止，而不仅仅是依靠所谓的逻辑。真正的村民往往会有更多的犹豫和不确定性，而狼人则会试图用看似合理的论点来掩盖他们的真正目的。
3. **加强合作**：我们需要更加紧密地合作，共同找出那些隐藏在我们中间的狼人。只有团结一致，我们才能取得最终的胜利。

昨晚的情况表明，我们需要更加谨慎地对待每一位玩家的言行。特别是那些试图主导讨论进程的玩家，我们需要更加敏锐地捕捉这些细节。昨晚7号玩家的死亡让我们不得不重新评估局势，我们需要更加紧密地合作，共同找出隐藏在我们中间的狼人。

在这里，我想特别关注4号玩家和5号玩家的发言。他们之前的发言虽然条理清晰，但似乎过于自信且显得有些刻意。特别是在他们强调要“关注逻辑推理”的时候，这种说法让我觉得有些刻意为之。虽然逻辑推理很重要，但在这样一个充满不确定性的游戏中，仅凭逻辑是不够的。我们需要更多地依赖于直觉和对细节的敏锐捕捉。

因此，我认为我们应该更加关注那些试图主导讨论进程的玩家，尤其是那些发言显得过于完美无缺的人。毕竟，在这样一个复杂的游戏中，

In [279]:
res_list

[1, 0, 2, 1, 1, 1]

In [566]:
def truncate(a):
    return a[:-1]


def speech(func):
    def wrapper(self,*args, **kwargs):
#         # 在调用原始函数之前可以执行一些代码
        result = func(*args, **kwargs)
        print(self.app)
        
#         # 对原始函数的输出进行后处理
#         result = result.upper()  # 假设我们想要将输出转换为大写
#         result = truncate(result)
        
#         # 在返回结果之前可以执行一些代码
#         print("处理后的结果是:", result)
        print(kwargs)
        
        print("hello,allice!")
        return result
    return wrapper

# 使用装饰器
class MyClass():
    def __init__(self):
        self.app=1
    @speech
    def greet(name,kk=0):
        return f"Hello, {name}!"

# 调用原始函数
cls=MyClass()
cls.greet("Alice",kk=1) # 输出: Hello, ALICE!

1
{'kk': 1}
hello,allice!


'Hello, Alice!'

In [268]:
class admin_logger():
    def __init__(self,db_name):
        self.conn=sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
    def find_all_games(self):
        sql="SELECT name FROM sqlite_master WHERE type='table'AND name NOT LIKE 'sqlite_%';"
        games = pd.read_sql_query(sql, self.conn)
        self.conn.commit()
        return games
    def drop_game(self,table_name):
        drop_query = f"DROP TABLE IF EXISTS {table_name};"
        self.cursor.execute(drop_query)
    def get_row_count(self, table_name):
        """
        查询指定表中的行数。
        
        :param table_name: 要查询的表的名称
        :return: 表中的行数
        """
        query = f"SELECT COUNT(*) FROM `{table_name}`;"
        try:
            self.cursor.execute(query)
            row_count = self.cursor.fetchone()[0]
            return row_count
        except sqlite3.Error as e:
            print(f"An error occurred: {e}")
            return None
    def check_logs(self,table_name):
        df = pd.read_sql_query(f"SELECT * FROM '{table_name}' ", self.conn)
        night=1
        day=0
        history_log="【游戏记录】:\n"+f"第 %d 夜:\n"%night
        for index, row in df.iterrows():
            if row['speaker']=='moderater':
                if row['content']=='夜晚结束' and index < len(df)-1:
                    day+=1
                    history_log+=f"第 %d 天:\n"%day
                elif row['content']=='白天结束' and index <len(df)-1:
                    night+=1
                    history_log+=f"第 %d 夜:\n"%night
                elif index == len(df)-1:
                    pass
                else:
                    formatted_log = f"发生事件: {row['content']}"
                    history_log+=formatted_log +"\n"
            else:
                formatted_log = f"{row['player']}号玩家执行 {row['type']}: {row['content']}"
                history_log+=formatted_log +"\n"
        return history_log

In [548]:
a=admin_logger('werewolf.db')
b=a.find_all_games()
b

,name
0,game_20240903220800
1,game_20240904081059
2,game_20240904180919
3,game_20240904182837
4,game_20240904214219
5,game_20240904220909
6,game_20240904222031
7,game_20240904223404
8,game_20240904223805
9,game_20240905223512


In [549]:
kk=sqlite3.connect('werewolf.db')
df = pd.read_sql_query(f"SELECT * FROM 'game_20240912074242' ", kk)
df.to_excel('123.xlsx')

In [546]:
c=a.check_logs('game_20240912074242')
print(c)

【游戏记录】:
第 1 夜:
发生事件: 游戏开始，当前存活玩家:0号玩家、1号玩家、2号玩家、3号玩家、4号玩家、5号玩家、6号玩家、7号玩家、
1号玩家执行 夜晚狼人杀人发言: 【发言内容】: 我们需要选择一个看起来可能是神职的玩家作为目标，优先考虑那些可能对局势有重大影响的角色。从目前的情况来看，我们可以先针对那些发言比较积极或者看起来可能带有重要技能的玩家。建议优先考虑3号或5号玩家，因为他们昨晚的行为显得有些可疑。我提议我们今晚先对3号玩家进行行动。 

（注：这里假定3号玩家在前一晚的行为引起了注意，实际游戏中应根据具体情况调整目标。）
2号玩家执行 夜晚狼人杀人发言: 【发言内容】: 同意1号的分析，3号玩家行为确实可疑，可能是神职。优先解决潜在威胁是明智之举。支持对3号玩家采取行动。
6号玩家执行 夜晚狼人杀人发言: 【发言内容】: 同意1号和2号的分析，3号玩家行为确实可疑。为了减少潜在威胁，我们应该优先对他采取行动。确认对3号玩家进行行动。
1号玩家执行 夜晚狼人杀人投票: 【投票结果】: 3号玩家
2号玩家执行 夜晚狼人杀人投票: 【投票结果】: 3号玩家
6号玩家执行 夜晚狼人杀人投票: 【投票结果】: 3号玩家
3号玩家执行 解药决策: 【解药决策】: 由于被消灭的玩家是自己（3号玩家），根据游戏策略的第一条指示，“如果被消灭的玩家是你自己，一定要使用解药”，因此我将使用解药自救。这样可以保持我的存活，继续为村民阵营提供帮助。 

使用解药。
3号玩家执行 毒药决策: 【毒药决策】: 在第一晚，由于我刚刚使用了解药自救，尚不清楚其他人的具体身份和局势。在这种情况下，盲目使用毒药可能会误杀无辜的村民或神职人员，反而削弱了我们阵营的力量。因此，在局势还不明朗的时候，谨慎起见，我决定今晚不使用毒药。

不使用毒药。
7号玩家执行 预言家验证身份: 在这种情况下，由于是第一晚，还没有足够的信息来判断谁最可疑。通常来说，在早期阶段，任何玩家都有可能是狼人。为了获取有用的信息，我会选择查验一个看起来比较关键或者直觉上感觉可疑的玩家。这里并没有前期的行为可以参考，因此选择可以基于位置或者其他非行为因素。

一种常见的策略是在早期查验可能的关键角色，比如可能会考虑查验一个玩家来试图尽早找出女巫或猎人，但在这个阶段，这样的策略未必有效，因为还不知道哪些玩家更

游戏策略的设计：
1.对于同一局游戏，优化策略分为三级：队伍策略/个人策略/行为方式；
    1-1.队伍策略描述角色之间合作的策略，队伍策略并不会直接发放给角色个人，而是由角色"coach"分配给角色；
    1-2.个人策略描述某个角色个人的行为策略，个人策略只会提示给角色作为战术指导，不会专门对行为进行约束；
    1-3.行为方式则是对对话、投票等方式的约束，需要通过@speech模块对玩家的发言、投票等行为进行检查和修改，行为方式不能和本局的个人策略冲突，甚至有部分行为准则需要由个人策略引申出来；

2.队伍策略：一般是互相配合的战略，需要由coach将团队策略分给同一团队的个人策略，其他方面与个人策略近似。
3.个人策略：个人行为策略，可以是长期战略（无行为准则支持，非硬性要求）。也可以是具体策略(应对某些情形时候的具体做法或者发言方案，或者某些战术)。
4.行为方式：具体对角色行为的规范，包括发言、投票以及技能的思考过程等。行为方式的要求比较具体，基于个人策略可以提取具体的行为方式及其要求。行为方式也可以以独立的形式给到角色。



#基本策略
狼人：白天发言：不要质疑自己队友；
     白天投票：不要投自己的队友；
预言家：夜晚验人:优先验证自己怀疑的对象
女巫：解药：1.自己被杀一定救自己；2.预言家一定救；3.快输了一定救；
     毒药：快输了一定用；
猎人：开枪：如果要输了，复仇自己怀疑的对象

In [547]:
#狼人：白天发言："为了提高狼人队友的存活率，不要在发言中质疑狼人阵营玩家的发言，减少大家投票消灭狼人阵营玩家的概率。"
#     晚上投票:"不要投票选择自己阵营的狼人玩家消灭。"
#预言家：夜晚验人："如果之前已经有发言记录，请分析大家的发言，选择一个你认为最可能是狼人的玩家进行验证。"
#女巫：解药："1.如果被消灭的玩家是你自己，一定要使用解药；2.如果你确定被消灭的玩家是预言家，可以考虑使用解药；3.根据胜利条件和当前剩余玩家的角色判断即将输掉比赛，请使用解药解救该玩家。"
#    毒药："根据胜利条件和当前剩余玩家的角色判断今晚即将输掉比赛，请根据游戏记录选择一个你认为最可能是狼人的玩家，对其使用毒药"
# 猎人：开枪："如果根据胜利条件和当前局势判断人类阵营即将输掉比赛，且已经有发言记录，请根据发言记录选择一个你认为最有可能是狼人的玩家复仇"   

In [271]:
#白天的语言都需要refine
#不同角色的思维策略都需要refine
#狼人晚上的发言不用优化
#投票过于集中，所有人都是统一的思考方式
#所有人都是通过对语言的分析来找狼人

In [272]:
#self-reflection
class reflector（）：
    def __init__(self,logger):
        self.llm=base_llm('你是一个优秀的狼人杀玩家')
        self.logger=logger
    def get_log(self,game_name):
        return self.logger.check_logs(game_name)
    def get_game_names(self):
        return self.logger.find_all_games()
    def self_reflection(self,log):
        #自我反省：通过对某局游戏进行检查，反思失败方为什么会失败。
        #step1：确定游戏玩家的角色和双方的胜利条件。
        #step2：总结整个游戏过程，以及胜利方的胜利过程。
        #step3：对失败方出现错误的环节进行分析(白天投票-白天发言、毒药策略、解药策略、猎枪策略、验人策略)
        #step4：给出优化建议（分析当前局面，可行的举措，执行原则）
        pass

In [273]:
#refine-module

In [274]:
#Evaluator

In [275]:
#self-reflector